In [6]:
import pandas as pd
import numpy as np
import time

from nba_api.stats.endpoints import playercareerstats, scoreboard, commonplayerinfo, boxscoreplayertrackv2, boxscorematchups

from nba_api.stats.static import teams,players

# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season, SeasonType, SeasonAll
from nba_api.stats.endpoints import playbyplay
import pickle

In [3]:
season_list = []
for year in range(3):
    season_list.append('202'+str(year)+'-2'+str(year+1))

In [ ]:
# Get all the NBA teams description
nba_teams = teams.get_teams()

season_campaign_all_teams_dict = {}
df_boxscores = pd.DataFrame()
count = 0
for selected_season in season_list:    
    for team_id in pd.DataFrame(nba_teams)['id']:
        time.sleep(1)
        campaign_all_teams_dict = {}
        print(team_id)
        campaign_all_teams_dict[team_id] = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id,
                                    season_nullable=selected_season,
                                    season_type_nullable=SeasonType.regular).get_data_frames()[0]

        campaign_all_teams_dict[team_id]['TEAM_ADVERSARY_ABBREVIATION'] = campaign_all_teams_dict[team_id]['MATCHUP'].apply(lambda x:x[-3:])
        campaign_all_teams_dict[team_id]['WINNER'] = campaign_all_teams_dict[team_id]['WL'].apply(lambda x:x == 'W')
        
        for game_id in campaign_all_teams_dict[team_id]['GAME_ID']:
            match_players_boxscore = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=game_id).get_data_frames()[0]
            selected_columns = ['GAME_ID','TEAM_ID','PLAYER_ID','PLAYER_NAME','START_POSITION']
            df = match_players_boxscore[selected_columns]
            df_boxscores = pd.concat([df_boxscores,match_players_boxscore[selected_columns]], ignore_index=True)

    season_campaign_all_teams_dict[selected_season] = campaign_all_teams_dict
    break

In [ ]:
# df_boxscores.query("GAME_ID == '0022000465' and TEAM_ID == 1610612738").drop_duplicates()
df_boxscores_no_duplicates = df_boxscores.drop_duplicates()

df_boxscores_no_duplicates.to_csv('players_formation.csv')

with open('season_campaign_all_teams_dict.pickle','wb') as results:
    pickle.dump(season_campaign_all_teams_dict, results)

df_boxscores_no_duplicates.head()
# season_campaign_all_teams_dict['2020-21'][1610612740]
# df_boxscores.query("GAME_ID == '0022000465'")
# df_boxscores.query("TEAM_ID == 1610612738")
# df_boxscores['GAME_ID'].value_counts()